In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

### Data preprocessing

In [2]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe("./data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("./data/green_tripdata_2021-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]  #'PULocationID', 'DOLocationID'
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

### Experiment and model tracking using MLflow

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/05/28 19:33:53 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1', creation_time=1716946433085, experiment_id='1', last_update_time=1716946433085, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    with open("models/lin_reg.bin", "wb") as f_out:
        pickle.dump((dv, lr), f_out)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [7]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "data/green_tripdata_2021-02.csv")

    alpha = 0.001
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    with open("models/lin_reg.bin", "wb") as f_out:
        pickle.dump((dv, lr), f_out)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

### Hyperparameter tuning using `hyperopt`

In [8]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}


search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),
    "learning_rate": hp.loguniform("learning_rate", -3, 0),
    "reg_alpha": hp.loguniform("reg_alpha", -5, -1),
    "reg_lambda": hp.loguniform("reg_lambda", -6, -1),
    "min_child_weight": hp.loguniform("min_child_weight", -1, 3),
    "objective": "reg:linear",
    "seed": 42,
}

best_result = fmin(
    fn=objective, space=search_space, algo=tpe.suggest, max_evals=50, trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:38:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.81771                          
[1]	validation-rmse:9.73330                           
[2]	validation-rmse:8.90401                           
[3]	validation-rmse:8.27454                           
[4]	validation-rmse:7.80557                           
[5]	validation-rmse:7.45606                           
[6]	validation-rmse:7.19836                           
[7]	validation-rmse:7.00741                           
[8]	validation-rmse:6.86772                           
[9]	validation-rmse:6.76382                           
[10]	validation-rmse:6.68554                          
[11]	validation-rmse:6.62870                          
[12]	validation-rmse:6.58378                          
[13]	validation-rmse:6.55326                          
[14]	validation-rmse:6.52708                          
[15]	validation-rmse:6.50538                          
[16]	validation-rmse:6.48631                          
[17]	validation-rmse:6.47321                          
[18]	valid

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:40:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.21176                                                        
[1]	validation-rmse:7.81396                                                        
[2]	validation-rmse:7.20278                                                        
[3]	validation-rmse:6.94047                                                        
[4]	validation-rmse:6.81784                                                        
[5]	validation-rmse:6.75300                                                        
[6]	validation-rmse:6.72307                                                        
[7]	validation-rmse:6.70526                                                        
[8]	validation-rmse:6.68651                                                        
[9]	validation-rmse:6.67539                                                        
[10]	validation-rmse:6.66977                                                       
[11]	validation-rmse:6.66648                                                

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:41:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.76090                                                      
[1]	validation-rmse:9.65578                                                       
[2]	validation-rmse:8.83629                                                       
[3]	validation-rmse:8.20670                                                       
[4]	validation-rmse:7.75759                                                       
[5]	validation-rmse:7.44659                                                       
[6]	validation-rmse:7.20110                                                       
[7]	validation-rmse:7.02637                                                       
[8]	validation-rmse:6.91100                                                       
[9]	validation-rmse:6.81614                                                       
[10]	validation-rmse:6.75773                                                      
[11]	validation-rmse:6.70609                                                      
[12]

KeyboardInterrupt: 

### Automatic logging using MLflow

In [9]:
params = {
    "learning_rate": 0.09585355369315604,
    "max_depth": 30,
    "min_child_weight": 1.060597050922164,
    "objective": "reg:linear",
    "reg_alpha": 0.018060244040060163,
    "reg_lambda": 0.011658731377413597,
    "seed": 42,
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50,
)

2024/05/28 19:43:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e994580ac9cb425880df95a23fb5edc7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:43:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

2024/05/28 19:45:44 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/28 19:45:44 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:45:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


In [10]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # log input data (DictVectorizers)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    # log model
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:46:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

2024/05/28 19:48:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:48:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:48:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


### Making predictions

In [11]:
logged_model = 'runs:/25329a592550460ab4004f056d268820/model'

# load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)

# load model as a XGBoost model
xgboost_model = mlflow.xgboost.load_model(logged_model)

y_pred = xgboost_model.predict(valid)

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:50:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


### Train multiple models for model registry

In [13]:
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
)
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    LinearSVR,
):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

2024/05/28 20:01:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 20:21:38 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 20:21:57 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/28 21:27:54 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY